In [74]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
%matplotlib inline
import glob

In [116]:
def read_data(classes):
    images = []
    labels = []
    
    for lbl in range(classes):
        full_path = os.path.join('GTSRB', 'Final_Training', 'Images') + '/' + format(lbl, '05d') + '/'
        
        print('reading folder: {}'.format(full_path))
        
        for img in glob.glob(full_path + '*.ppm'):
            
            image = plt.imread(img).astype(np.float32)
            
            label = np.zeros((classes, ), dtype = np.float32)
            label[lbl] = 1.0
            
            images.append(img)
            labels.append(label)
            
    return images, labels

In [117]:
num_classes = 43
images, labels = read_data(num_classes)

reading folder: GTSRB/Final_Training/Images/00000/
reading folder: GTSRB/Final_Training/Images/00001/
reading folder: GTSRB/Final_Training/Images/00002/
reading folder: GTSRB/Final_Training/Images/00003/
reading folder: GTSRB/Final_Training/Images/00004/
reading folder: GTSRB/Final_Training/Images/00005/
reading folder: GTSRB/Final_Training/Images/00006/
reading folder: GTSRB/Final_Training/Images/00007/
reading folder: GTSRB/Final_Training/Images/00008/
reading folder: GTSRB/Final_Training/Images/00009/
reading folder: GTSRB/Final_Training/Images/00010/
reading folder: GTSRB/Final_Training/Images/00011/
reading folder: GTSRB/Final_Training/Images/00012/
reading folder: GTSRB/Final_Training/Images/00013/
reading folder: GTSRB/Final_Training/Images/00014/
reading folder: GTSRB/Final_Training/Images/00015/
reading folder: GTSRB/Final_Training/Images/00016/
reading folder: GTSRB/Final_Training/Images/00017/
reading folder: GTSRB/Final_Training/Images/00018/
reading folder: GTSRB/Final_Tra